Karthick Ramasubramanian KR33733

## Task A

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [ ]:
url='https://www.beeradvocate.com/beer/top-rated/'
wd= webdriver.Chrome('chromedriver',chrome_options=chrome_options)
wd.get(url)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: use options instead of chrome_options
  


In [ ]:
# beer_links=wd.find_elements_by_xpath("//td[@class='hr_bottom_light']/a")

def get_beer_links():
  url='https://www.beeradvocate.com/beer/top-rated/'
  wd= webdriver.Chrome('chromedriver',chrome_options=chrome_options)
  wd.get(url)
  beer_links=wd.find_elements_by_xpath("//td[@class='hr_bottom_light']/a")
  return (beer_links,wd)

def extract_review_info(brand):
  def extract_text(e):
    lt_temp=np.array(e.split('\n\n'))
    # print(lt_temp)
    rating=lt_temp[1]
    date=lt_temp[-1]

    lt_temp=np.delete(lt_temp,[0,1,-1])

    return ' '.join(lt_temp)

  scr_x_path="//div[contains(@id,'rating_fullview_content')]/span[@class='BAscore_norm']"
  txt_x_path="//div[contains(@id,'rating_fullview_content')]"

  rating_elements=wd.find_elements_by_xpath(scr_x_path)
  txt_elements=wd.find_elements_by_xpath(txt_x_path)





  txt_reviews=[extract_text(i.text) for i in txt_elements]
  rating_reviews=[i.text for i in rating_elements ]

  return pd.DataFrame({'brand_name':brand,
                       'rating':rating_reviews,
                        'review':txt_reviews})
  


In [ ]:
# temp_links,wd=get_beer_links()#need to create and close webdriver everytime as the back navigator is not working
# link=temp_links[3]

# time.sleep(1)
# link.click()

wd.close()

In [ ]:
# link.href
link.get_attribute('href') 

'https://www.beeradvocate.com/beer/profile/25888/87246/'

In [ ]:
# df_info=pd.DataFrame()

# ct=0
while ct<250:

  temp_links,wd=get_beer_links()#need to create and close webdriver everytime as the back navigator is not working
  link=temp_links[ct]



  brand=link.text

  try:
    link.click()
  except:
    new_url=link.get_attribute('href')#new url
    wd.close()
    print('page closed')

    wd= webdriver.Chrome('chromedriver',chrome_options=chrome_options)
    wd.get(new_url)

  temp=extract_review_info(brand)

  df_info=pd.concat([df_info,temp])

  wd.close()
  ct+=1
  print(ct)

  # wd.execute_script("window.history.go(-1)")
  # wd.back()

In [ ]:
df_info

,brand_name,rating,review
0,Kentucky Brunch Brand Stout,4.56,"Long time waiting to tick this one, and I have..."
1,Kentucky Brunch Brand Stout,5,This review is for the 2019 batch. It was bott...
2,Kentucky Brunch Brand Stout,5,Supreme maple OD! Soooo easy drinking & well-t...
3,Kentucky Brunch Brand Stout,5,I have now had 4 different years of KBBS and c...
4,Kentucky Brunch Brand Stout,5,2020 Bottle. Absolutely bonkers Maple Syrup on...
...,...,...,...
20,Dragonsaddle,4.75,"For me, I have experienced all of the hoof hea..."
21,Dragonsaddle,4.5,fresh can drank on 11/5/2016 very hazy golden ...
22,Dragonsaddle,4.4,1 PINT can\nServed in an oversized stemless wi...
23,Dragonsaddle,4.79,L: Pours a dark gold with a nice head that sta...


In [ ]:
df_info.to_csv('data.csv')

In [ ]:
pd.DataFrame({'rating':rating_reviews,
              'review':txt_reviews})
# pd.DataFrame(txt_reviews[0])

In [ ]:
wd.close()

## Task B : Getting Attributes

In [3]:
import re
import spacy
import nltk

import nltk
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
from nltk.tokenize import word_tokenize

# import gensim
# from gensim import corpora

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

from string import punctuation
stop_words = set( stopwords.words('english'))

%matplotlib inline

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hatricano\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Hatricano\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hatricano\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
# from google.colab import files 
# files.upload()
df = pd.read_csv('data.csv')

In [5]:
df.head()

,Unnamed: 0,brand_name,rating,review
0,0,Kentucky Brunch Brand Stout,4.56,"Long time waiting to tick this one, and I have..."
1,1,Kentucky Brunch Brand Stout,5.00,This review is for the 2019 batch. It was bott...
2,2,Kentucky Brunch Brand Stout,5.00,Supreme maple OD! Soooo easy drinking & well-t...
3,3,Kentucky Brunch Brand Stout,5.00,I have now had 4 different years of KBBS and c...
4,4,Kentucky Brunch Brand Stout,5.00,2020 Bottle. Absolutely bonkers Maple Syrup on...


In [6]:
# stop loss words 
stop = set(stopwords.words('english'))

# punctuation 
exclude = set(string.punctuation) 

# lemmatization
lemma = WordNetLemmatizer() 


# One function for all the steps:
def clean(doc):
    # convert text into lower case + split into words
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    # remove any stop words present
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)  
    # remove punctuations + normalize the text
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())  
    
    return normalized

In [7]:
df['clean'] = df['review'].map(clean)
df['review_clean_tokanize'] = df['clean'].apply(word_tokenize)

In [11]:
## Word Count from Review

from collections import defaultdict
word_count=defaultdict(int)

ct=0

for i in range(len(df)):
    for words in df['review_clean_tokanize'][i]:
        word_count[words] +=1

word_ct_sorted={i:j for i,j in sorted(word_count.items(), key=lambda d:d[1],reverse=True)}
# word_ct_sorted

## Part C: Similarity

In [12]:
# Aggressive (Boldly assertive aroma and/or taste) 
# Balanced: Malt and hops in similar proportions; equal representation of malt sweetness and hop bitterness in the flavor — especially at the finish
# Complex: Multidimensional; many flavors and sensations on the palate
# Crisp: Highly carbonated; effervescentFruity: Flavors reminiscent of various fruits 
# Hoppy: Herbal, earthy, spicy, or citric aromas and flavors of hops or Malty: Grainy, caramel-like; can be sweet or dry
# Robust: Rich and full-bodied
# Diacetyl: Buttery or butterscotchy aroma or flavor
# Estery: Fruity aromas
# Floral: Full of aromas reminiscent of flowers
# Fruity: Flavors reminiscent of various fruits
# Malty: Grainy, caramel-like; can be sweet or dry
# Roasty/toasty: Malt (roasted grain) flavors

In [13]:
customer_attribute = pd.DataFrame(index=['aggressive','balanced','complex','crisp','hoppy','robust'], columns=['description'])
customer_attribute.loc['aggressive'] = 'Malt and hops in similar proportions; equal representation of malt sweetness and hop bitterness in the flavor — especially at the finish'
customer_attribute.loc['balanced'] = 'Multidimensional; many flavors and sensations on the palate'
customer_attribute.loc['complex'] = 'Boldly assertive aroma and/or taste'
customer_attribute.loc['crisp'] = 'Highly carbonated; effervescent Fruity: Flavors reminiscent of various fruits'
customer_attribute.loc['hoppy'] = 'Herbal, earthy, spicy, or citric aromas and flavors of hops or Malty: Grainy, caramel-like; can be sweet or dry'
customer_attribute.loc['robust'] = 'Rich and full-bodied'
customer_attribute.loc['diacetyl'] = 'Buttery or butterscotchy aroma or flavor'
customer_attribute.loc['estery'] = 'Fruity aromas'
customer_attribute.loc['floral'] = 'Full of aromas reminiscent of flowers'
customer_attribute.loc['fruity'] = 'Flavors reminiscent of various fruits'
customer_attribute.loc['malty'] = 'Grainy, caramel-like; can be sweet or dry'
customer_attribute.loc['roasty/toasty'] = 'Malt (roasted grain) flavors'

customer_attribute['clean'] = customer_attribute['description'].map(clean)
customer_attribute

,description,clean
aggressive,Malt and hops in similar proportions; equal re...,malt hop similar proportion equal representati...
balanced,Multidimensional; many flavors and sensations ...,multidimensional many flavor sensation palate
complex,Boldly assertive aroma and/or taste,boldly assertive aroma andor taste
crisp,Highly carbonated; effervescent Fruity: Flavor...,highly carbonated effervescent fruity flavor r...
hoppy,"Herbal, earthy, spicy, or citric aromas and fl...",herbal earthy spicy citric aroma flavor hop ma...
robust,Rich and full-bodied,rich fullbodied
diacetyl,Buttery or butterscotchy aroma or flavor,buttery butterscotchy aroma flavor
estery,Fruity aromas,fruity aroma
floral,Full of aromas reminiscent of flowers,full aroma reminiscent flower
fruity,Flavors reminiscent of various fruits,flavor reminiscent various fruit


In [19]:
frame = [customer_attribute[['clean']],df[['clean']] ]
attribute_and_reviews = pd.concat(frame,join='outer')

count_vec = CountVectorizer()
word_vector = count_vec.fit_transform(attribute_and_reviews['clean']).toarray()

attribute = word_vector[0:12,:]
reviews = word_vector[12:,]

In [23]:
similarity = pd.DataFrame(cosine_similarity(X = reviews, Y=attribute), index = df.index, columns=customer_attribute.index )
similarity['brand_name'],similarity['rating'],similarity['review'] = df['brand_name'], df['rating'], df['review']
similarity.head()

,aggressive,balanced,complex,crisp,hoppy,robust,diacetyl,estery,floral,fruity,malty,roasty/toasty,brand_name,rating,review
0,0.046676,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,Kentucky Brunch Brand Stout,4.56,"Long time waiting to tick this one, and I have..."
1,0.033634,0.062017,0.000000,0.049029,0.040032,0.0,0.069338,0.0,0.069338,0.069338,0.0,0.069338,Kentucky Brunch Brand Stout,5.00,This review is for the 2019 batch. It was bott...
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,Kentucky Brunch Brand Stout,5.00,Supreme maple OD! Soooo easy drinking & well-t...
3,0.000000,0.000000,0.062017,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,Kentucky Brunch Brand Stout,5.00,I have now had 4 different years of KBBS and c...
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,Kentucky Brunch Brand Stout,5.00,2020 Bottle. Absolutely bonkers Maple Syrup on...


In [31]:
# Customer Preference Input
# Choose 3 attribute from : 
# Aggressive, Balanced, Complex, Crisp, Hoppy, Robust, Diacetyl, Estery, Floral, Fruity, Malty, Roasty/toasty
a = input("Enter 1st Attribute:").lower()
b = input("Enter 2nd Attribute:").lower()
c = input("Enter 3rd Attribute:").lower()
preference_list=[a,b,c]

Enter 1st Attribute:Complex
Enter 2nd Attribute:Crisp
Enter 3rd Attribute:Hoppy


In [44]:
#Output File 
columns = ['brand_name','review'] + preference_list
attribute_similarity = similarity[columns]
attribute_similarity.head()

,brand_name,review,complex,crisp,hoppy
0,Kentucky Brunch Brand Stout,"Long time waiting to tick this one, and I have...",0.000000,0.000000,0.000000
1,Kentucky Brunch Brand Stout,This review is for the 2019 batch. It was bott...,0.000000,0.049029,0.040032
2,Kentucky Brunch Brand Stout,Supreme maple OD! Soooo easy drinking & well-t...,0.000000,0.000000,0.000000
3,Kentucky Brunch Brand Stout,I have now had 4 different years of KBBS and c...,0.062017,0.000000,0.000000
4,Kentucky Brunch Brand Stout,2020 Bottle. Absolutely bonkers Maple Syrup on...,0.000000,0.000000,0.000000


In [47]:
# Brand Attribute average similarity
attribute_similarity[['brand_name','complex','crisp','hoppy']].groupby(by=['brand_name']).mean()

,complex,crisp,hoppy
brand_name,,,
A Deal With The Devil,0.056678,0.040272,0.071706
A Deal With The Devil - Double Oak-Aged,0.035518,0.049384,0.054692
Aaron,0.041683,0.042808,0.060647
Abner,0.062675,0.036660,0.065511
Abrasive Ale,0.060460,0.045823,0.088004
...,...,...,...
Westly,0.042772,0.076247,0.037869
Wide Awake It's Morning,0.064278,0.018977,0.055755
Zenne Y Frontera,0.044155,0.034615,0.039475


## Part D:  Sentiment Analysis

In [33]:
# !pip install vaderSentiment

In [48]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

def get_sentiment(review, **kwargs):
    sentiment_score = analyser.polarity_scores(review)
    positive_meter = round((sentiment_score['pos']), 2)
    negative_meter = round((sentiment_score['neg']), 2)
    if positive_meter >= negative_meter:
        return positive_meter
    else:
        return -1 * negative_meter

In [49]:
df['sentiment_status'] = df['clean'].apply(get_sentiment)

In [50]:
# Negative sentiments are -ve. 
df.head()

,Unnamed: 0,brand_name,rating,review,clean,review_clean_tokanize,sentiment_status
0,0,Kentucky Brunch Brand Stout,4.56,"Long time waiting to tick this one, and I have...",long time waiting tick one say pretty underwhe...,"[long, time, waiting, tick, one, say, pretty, ...",0.31
1,1,Kentucky Brunch Brand Stout,5.00,This review is for the 2019 batch. It was bott...,review 2019 batch bottle number 43 tasted part...,"[review, 2019, batch, bottle, number, 43, tast...",0.23
2,2,Kentucky Brunch Brand Stout,5.00,Supreme maple OD! Soooo easy drinking & well-t...,supreme maple od soooo easy drinking welltasty...,"[supreme, maple, od, soooo, easy, drinking, we...",0.36
3,3,Kentucky Brunch Brand Stout,5.00,I have now had 4 different years of KBBS and c...,4 different year kbbs continue feel beer realm...,"[4, different, year, kbbs, continue, feel, bee...",0.25
4,4,Kentucky Brunch Brand Stout,5.00,2020 Bottle. Absolutely bonkers Maple Syrup on...,2020 bottle absolutely bonkers maple syrup nos...,"[2020, bottle, absolutely, bonkers, maple, syr...",-0.14


## Part E:  Product Recommendation

In [51]:
# Using the same customer Preference
preference_list

['complex', 'crisp', 'hoppy']

In [52]:
# 
columns2 = ['brand_name'] + preference_list
customer_choice = similarity[columns2]
customer_choice['customer_preference'] = customer_choice[preference_list[0]]/3 + customer_choice[preference_list[1]]/3 + customer_choice[preference_list[2]]/3
beer_rating = customer_choice[['brand_name','customer_preference']].groupby(by= 'brand_name').mean()

<ipython-input-52-90e62118c12c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customer_choice['customer_preference'] = customer_choice[preference_list[0]]/3 + customer_choice[preference_list[1]]/3 + customer_choice[preference_list[2]]/3


In [56]:
beer_rating['sentiment_status'] = df[['brand_name', 'sentiment_status']].groupby(by = 'brand_name').mean().sort_values(by='sentiment_status')['sentiment_status']

In [87]:
#Overall Recommendation

#Normalisation of sentiment and preference
cpm = customer_choice['customer_preference'].min()
cpn = customer_choice['customer_preference'].max() - customer_choice['customer_preference'].min()
sm = df['sentiment_status'].min()
sn = df['sentiment_status'].max() - df['sentiment_status'].min()


beer_rating['overall_rating'] = (beer_rating['customer_preference'] - cpm)/cpn + (beer_rating['sentiment_status'] - sm)/sn
beer_rating.head()

,customer_preference,sentiment_status,overall_rating
brand_name,,,
A Deal With The Devil,0.056219,0.2704,0.808906
A Deal With The Devil - Double Oak-Aged,0.046531,0.3216,0.810413
Aaron,0.048379,0.2308,0.743705
Abner,0.054949,0.2272,0.768191
Abrasive Ale,0.064762,0.2448,0.823605


In [88]:
# Product Recommendation
beer_rating.sort_values(by='overall_rating',ascending = False).index[:3].values

array(['King Sue', 'Miami Madness', 'DFPF'], dtype=object)

In [109]:
# Sentiment and Similarity Score (Overall rating is normalised)
beer_rating.loc[beer_rating.sort_values(by='overall_rating',ascending = False).index[:3]]

,customer_preference,sentiment_status,overall_rating
brand_name,,,
King Sue,0.082866,0.2572,0.909385
Miami Madness,0.070487,0.3116,0.902272
DFPF,0.061469,0.3404,0.888212


## Part F: Word Embeddings - using Glove

In [89]:
from scipy import spatial

In [91]:
embeddings_dict = {}
with open("glove.6B.300d.txt", 'r',encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[word] = vector

In [92]:
# preference token
customer_pref_tokens = customer_attribute['clean'].apply(word_tokenize).loc['crisp']+customer_attribute['clean'].apply(word_tokenize).loc['hoppy']

customer_pref_tokens[:5]

['highly', 'carbonated', 'effervescent', 'fruity', 'flavor']

In [93]:
def review_cosine(list1):
    
    customer_pref_tokens_embeddings = np.array([0.00]*300) 
    review_embeddings = np.array([0.00]*300)
    
    try:
        for i in customer_pref_tokens:
            customer_pref_tokens_embeddings += embeddings_dict[i.lower()]
    except:
        x = 0 # do nothing
        
    customer_pref_tokens_embeddings = customer_pref_tokens_embeddings/len(customer_pref_tokens)
        
    try:
        for i in list1:
            review_embeddings += embeddings_dict[i.lower()]
    except:
        x=0 # do nothing
    
    review_embeddings = review_embeddings/len(list1)
    
    return (1 - spatial.distance.cosine(customer_pref_tokens_embeddings, review_embeddings))
    

In [94]:
df['word_embedding_cosine_similarity_attribute_and_review'] = df['review_clean_tokanize'].map(review_cosine)
df.head()

C:\ProgramData\Anaconda3\lib\site-packages\scipy\spatial\distance.py:714: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


,Unnamed: 0,brand_name,rating,review,clean,review_clean_tokanize,sentiment_status,word_embedding_cosine_similarity_attribute_and_review
0,0,Kentucky Brunch Brand Stout,4.56,"Long time waiting to tick this one, and I have...",long time waiting tick one say pretty underwhe...,"[long, time, waiting, tick, one, say, pretty, ...",0.31,0.345601
1,1,Kentucky Brunch Brand Stout,5.00,This review is for the 2019 batch. It was bott...,review 2019 batch bottle number 43 tasted part...,"[review, 2019, batch, bottle, number, 43, tast...",0.23,0.461112
2,2,Kentucky Brunch Brand Stout,5.00,Supreme maple OD! Soooo easy drinking & well-t...,supreme maple od soooo easy drinking welltasty...,"[supreme, maple, od, soooo, easy, drinking, we...",0.36,0.259267
3,3,Kentucky Brunch Brand Stout,5.00,I have now had 4 different years of KBBS and c...,4 different year kbbs continue feel beer realm...,"[4, different, year, kbbs, continue, feel, bee...",0.25,0.109184
4,4,Kentucky Brunch Brand Stout,5.00,2020 Bottle. Absolutely bonkers Maple Syrup on...,2020 bottle absolutely bonkers maple syrup nos...,"[2020, bottle, absolutely, bonkers, maple, syr...",-0.14,0.416121


In [106]:
#Overall Top 3 Recommendation

beer_rating_embedding = df[['brand_name','word_embedding_cosine_similarity_attribute_and_review']].groupby(by= 'brand_name').mean()
beer_rating_embedding['sentiment_status'] = df[['brand_name', 'sentiment_status']].groupby(by = 'brand_name').mean()

# Normalised
wer =  beer_rating_embedding['word_embedding_cosine_similarity_attribute_and_review'].max()-beer_rating_embedding['word_embedding_cosine_similarity_attribute_and_review'].min()
wem = beer_rating_embedding['word_embedding_cosine_similarity_attribute_and_review'].min()

sm = beer_rating_embedding['sentiment_status'].min()
sr = beer_rating_embedding['sentiment_status'].max() - beer_rating_embedding['sentiment_status'].min()

beer_rating_embedding['overall_rating'] = (beer_rating_embedding['word_embedding_cosine_similarity_attribute_and_review']-wem)/wer + (beer_rating_embedding['sentiment_status']-sm)/sr
beer_rating_embedding.sort_values(by='overall_rating',ascending = False).index[:3].values

array(['Framboise Du Fermier', 'Breakfast Stout',
       'Speedway Stout - Vietnamese Coffee'], dtype=object)

In [ ]:
# The recommendation is different from using WordEmbeddings and Bag of Words

In [153]:
# Cosine Similarity Comparision between Bag of Words and Using Word Embeddings.
WE_BoW_compare = pd.DataFrame()
WE_BoW_compare[['Word_Embedding_cosine_similarity']] = beer_rating_embedding[['word_embedding_cosine_similarity_attribute_and_review']]
WE_BoW_compare['bag_of_words_cosine_similarity'] = beer_rating['customer_preference']
WE_BoW_compare.loc[beer_rating_embedding.sort_values(by='overall_rating',ascending = False).index[:3]]

,Word_Embedding_cosine_similarity,bag_of_words_cosine_similarity
brand_name,,
Framboise Du Fermier,0.505260,0.034469
Breakfast Stout,0.485395,0.043470
Speedway Stout - Vietnamese Coffee,0.529738,0.045570


In [155]:
# Normalised Cosine Similarity Comparision
wem = WE_BoW_compare['Word_Embedding_cosine_similarity'].min()
wer = WE_BoW_compare['Word_Embedding_cosine_similarity'].max() - WE_BoW_compare['Word_Embedding_cosine_similarity'].min()

bowm = WE_BoW_compare['bag_of_words_cosine_similarity'].min()
bowr = WE_BoW_compare['bag_of_words_cosine_similarity'].max() - WE_BoW_compare['bag_of_words_cosine_similarity'].min()

WE_BoW_compare['bag_of_words_cosine_similarity'] = (WE_BoW_compare['bag_of_words_cosine_similarity']- bowm)/bowr
WE_BoW_compare['Word_Embedding_cosine_similarity'] = (WE_BoW_compare['Word_Embedding_cosine_similarity'] - wem)/wer

WE_BoW_compare.loc[beer_rating_embedding.sort_values(by='overall_rating',ascending = False).index[:3]]

,Word_Embedding_cosine_similarity,bag_of_words_cosine_similarity
brand_name,,
Framboise Du Fermier,0.809027,0.183745
Breakfast Stout,0.744754,0.335559
Speedway Stout - Vietnamese Coffee,0.888223,0.370973


In [ ]:
# We Observe that the Cosine similarity of the products using the 2 methods is very different.

## Part G: Top 3 Products

In [111]:
# This recommendation will be the ones which have the attributes that are most amoung the general population. 
# For customers looking for more nuanced attributes, the recommendation will be different.

In [113]:
# Top Beers Based on sentiment and Attribute preference
beer_rating.loc[beer_rating.sort_values(by='overall_rating',ascending = False).index[:3]]

,customer_preference,sentiment_status,overall_rating
brand_name,,,
King Sue,0.082866,0.2572,0.909385
Miami Madness,0.070487,0.3116,0.902272
DFPF,0.061469,0.3404,0.888212


In [121]:
# Top beers based on Rating
beer_rating.loc[df[['rating','brand_name']].groupby(by=['brand_name']).mean().sort_values(by = 'rating',ascending=False).index[:3]]

,customer_preference,sentiment_status,overall_rating
brand_name,,,
Chemtrailmix,0.023574,0.259200,0.663380
Vanilla Bean Assassin,0.032558,0.274375,0.713339
Blessed,0.035185,0.299200,0.744663


In [ ]:
# Data shows that customer preference is very low in top 3 by ratings beer.